In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from fairseq.data import data_utils
from fairseq.data.dictionary import Dictionary
from fairseq.models.roberta import RobertaModel
import argparse
import pathlib

import fairseq
import torch
from fairseq.models.roberta import RobertaModel as FairseqRobertaModel
from fairseq.modules import TransformerSentenceEncoderLayer
from packaging import version

from transformers import RobertaConfig, AutoTokenizer
import os
if 0:#os.getenv("PRELAYERNORM") is not None and os.getenv("PRELAYERNORM") == "1":
    print("Use pre layer norm")
    import sys
    sys.path.append("/mnt/nfs-data/users/anhvth5/DinkyTrain")
    from huggingface.modeling_roberta_prelayernorm import RobertaForMaskedLM, RobertaForSequenceClassification
    pre_layer_norm = True
else:
    from transformers import RobertaModel as TransformerRobertaModel
    pre_layer_norm = False
# from convert_dsfs_ckpt_to_fs_ckpt import convert_dsfs_ckpt_to_fs_ckpt
from transformers.utils import logging
import shutil
import tempfile


# Convert model

In [3]:
ls ../store/vi-roberta-base/

bert_spm.model            checkpoint_177_105000.pt  checkpoint_48_30000.pt
checkpoint_108_65000.pt   checkpoint_17_10000.pt    checkpoint_56_35000.pt
checkpoint_116_70000.pt   checkpoint_185_110000.pt  checkpoint_73_45000.pt
checkpoint_125_75000.pt   checkpoint_194_115000.pt  checkpoint_99_60000.pt
checkpoint_134_80000.pt   checkpoint_203_120000.pt  checkpoint_9_5000.pt
checkpoint_142_85000.pt   checkpoint_211_125000.pt  checkpoint_last.pt
checkpoint_151_90000.pt   checkpoint_25_15000.pt    checkpoint_last.pt.tmp
checkpoint_159_95000.pt   checkpoint_33_20000.pt    dict.txt
checkpoint_168_100000.pt  checkpoint_40_25000.pt    test.py


In [20]:
from fairseq.models.roberta import RobertaModel
roberta  = RobertaModel.from_pretrained('../store/vi-roberta-base/', 'checkpoint_211_125000.pt',bpe='sentencepiece', 
                                                sentencepiece_model='../store/bert_spm.model')
roberta.eval();
roberta_sent_encoder = roberta.model.encoder.sentence_encoder

2023-10-09 09:26:12 | INFO | fairseq.file_utils | loading archive file ../store/vi-roberta-base/
2023-10-09 09:26:13 | INFO | fairseq.tasks.masked_lm | dictionary: 98584 types
2023-10-09 09:26:16 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 10, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1,

In [24]:
hf_model_config = "roberta-base"

config = RobertaConfig.from_pretrained(hf_model_config)
config.vocab_size = 98585
if classification_head:
    config.num_labels = roberta.model.classification_heads["mnli"].out_proj.weight.shape[0]
print("Our RoBERTa config:", config)

model = RobertaForSequenceClassification(config) if classification_head else RobertaForMaskedLM(config)
model.eval()

Our RoBERTa config: RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 98585
}



RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(98585, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [25]:
pre_layer_norm = False
classification_head = False
# Now let's copy all the weights.
# Embeddings
model.roberta.embeddings.word_embeddings.weight = roberta_sent_encoder.embed_tokens.weight
model.roberta.embeddings.position_embeddings.weight = roberta_sent_encoder.embed_positions.weight
model.roberta.embeddings.token_type_embeddings.weight.data = torch.zeros_like(
    model.roberta.embeddings.token_type_embeddings.weight
)  # just zero them out b/c RoBERTa doesn't use them.
model.roberta.embeddings.LayerNorm.weight = roberta_sent_encoder.layernorm_embedding.weight
model.roberta.embeddings.LayerNorm.bias = roberta_sent_encoder.layernorm_embedding.bias

for i in range(config.num_hidden_layers):
    # Encoder: start of layer
    layer = model.roberta.encoder.layer[i]
    roberta_layer: TransformerSentenceEncoderLayer = roberta_sent_encoder.layers[i]

    # self attention
    self_attn = layer.attention.self
    assert (
        roberta_layer.self_attn.k_proj.weight.data.shape
        == roberta_layer.self_attn.q_proj.weight.data.shape
        == roberta_layer.self_attn.v_proj.weight.data.shape
        == torch.Size((config.hidden_size, config.hidden_size))
    )

    self_attn.query.weight.data = roberta_layer.self_attn.q_proj.weight
    self_attn.query.bias.data = roberta_layer.self_attn.q_proj.bias
    self_attn.key.weight.data = roberta_layer.self_attn.k_proj.weight
    self_attn.key.bias.data = roberta_layer.self_attn.k_proj.bias
    self_attn.value.weight.data = roberta_layer.self_attn.v_proj.weight
    self_attn.value.bias.data = roberta_layer.self_attn.v_proj.bias

    # self-attention output
    self_output = layer.attention.output
    assert self_output.dense.weight.shape == roberta_layer.self_attn.out_proj.weight.shape
    self_output.dense.weight = roberta_layer.self_attn.out_proj.weight
    self_output.dense.bias = roberta_layer.self_attn.out_proj.bias
    if pre_layer_norm:
        layer.attention.LayerNorm.weight = roberta_layer.self_attn_layer_norm.weight
        layer.attention.LayerNorm.bias = roberta_layer.self_attn_layer_norm.bias
    else:
        self_output.LayerNorm.weight = roberta_layer.self_attn_layer_norm.weight
        self_output.LayerNorm.bias = roberta_layer.self_attn_layer_norm.bias

    # intermediate
    intermediate = layer.intermediate
    assert intermediate.dense.weight.shape == roberta_layer.fc1.weight.shape
    intermediate.dense.weight = roberta_layer.fc1.weight
    intermediate.dense.bias = roberta_layer.fc1.bias

    # output
    bert_output = layer.output
    assert bert_output.dense.weight.shape == roberta_layer.fc2.weight.shape
    bert_output.dense.weight = roberta_layer.fc2.weight
    bert_output.dense.bias = roberta_layer.fc2.bias
    if pre_layer_norm:
        layer.intermediate.LayerNorm.weight = roberta_layer.final_layer_norm.weight
        layer.intermediate.LayerNorm.bias = roberta_layer.final_layer_norm.bias
    else:
        bert_output.LayerNorm.weight = roberta_layer.final_layer_norm.weight
        bert_output.LayerNorm.bias = roberta_layer.final_layer_norm.bias

    # end of layer

# The last layer norm layer for pre-layernorm  
if pre_layer_norm:
    model.roberta.LayerNorm.weight = roberta_sent_encoder.layer_norm.weight
    model.roberta.LayerNorm.bias = roberta_sent_encoder.layer_norm.bias

if classification_head:
    model.classifier.dense.weight = roberta.model.classification_heads["mnli"].dense.weight
    model.classifier.dense.bias = roberta.model.classification_heads["mnli"].dense.bias
    model.classifier.out_proj.weight = roberta.model.classification_heads["mnli"].out_proj.weight
    model.classifier.out_proj.bias = roberta.model.classification_heads["mnli"].out_proj.bias
else:
    # LM Head
    model.lm_head.dense.weight = roberta.model.encoder.lm_head.dense.weight
    model.lm_head.dense.bias = roberta.model.encoder.lm_head.dense.bias
    model.lm_head.layer_norm.weight = roberta.model.encoder.lm_head.layer_norm.weight
    model.lm_head.layer_norm.bias = roberta.model.encoder.lm_head.layer_norm.bias
    model.lm_head.decoder.weight = roberta.model.encoder.lm_head.weight
    model.lm_head.decoder.bias = roberta.model.encoder.lm_head.bias
    model.lm_head.bias = roberta.model.encoder.lm_head.bias


In [26]:
# model.embeddings

In [27]:
SAMPLE_TEXT = 'xin chào'
hf_ckpt_path = '.cache/hf_ckpt_path'

In [28]:
# Let's check that we get the same results.
input_ids: torch.Tensor = roberta.encode(SAMPLE_TEXT).unsqueeze(0)  # batch of size 1
our_output = model(input_ids, output_hidden_states=True)
our_output_final = our_output[0].cpu()
our_output = our_output.hidden_states 
if classification_head:
    their_output = roberta.model.classification_heads["mnli"](roberta.extract_features(input_ids))
else:
    their_output = roberta.model(input_ids, return_all_hiddens=True)
    their_output_final = their_output[0].cpu()
    their_output = their_output[1]['inner_states'] 

# for i in range(len(our_output)):
#     max_absolute_diff = torch.max(torch.abs(our_output[i].cpu() - their_output[i].transpose(0, 1).cpu())).item()
#     mean_absolute_diff = torch.mean(torch.abs(our_output[i].cpu() - their_output[i].transpose(0, 1).cpu())).item()
#     print("Layer %d max diff: %f mean diff: %f" % (i, max_absolute_diff, mean_absolute_diff)) 

max_absolute_diff = torch.max(torch.abs(our_output_final - their_output_final)).item()
print(f"max_absolute_diff = {max_absolute_diff}") 
success = torch.allclose(our_output_final, their_output_final, atol=1e-1)
print("Do both models output the same tensors?", "🔥" if success else "💩")
if not success:
    raise Exception("Something went wRoNg")
pathlib.Path(hf_ckpt_path).mkdir(parents=True, exist_ok=True)
print(f"Saving model to {hf_ckpt_path}")
model.save_pretrained(hf_ckpt_path)
# tokenizer = AutoTokenizer.from_pretrained(hf_model_config)
# tokenizer.save_pretrained(hf_ckpt_path)

max_absolute_diff = 2.09808349609375e-05
Do both models output the same tensors? 🔥
Saving model to .cache/hf_ckpt_path


In [29]:
from transformers import AutoModel
hf = AutoModel.from_pretrained('.cache/hf_ckpt_path/').eval()

Some weights of RobertaModel were not initialized from the model checkpoint at .cache/hf_ckpt_path/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
ids = roberta.encode('xin chào')
r_features = roberta.extract_features(ids)

In [31]:

h_features = hf.forward(ids[None]).last_hidden_state


In [32]:
r_features.shape, h_features.shape

(torch.Size([1, 4, 768]), torch.Size([1, 4, 768]))

In [33]:
torch.isclose(r_features, h_features, 1e-4)

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [34]:
torch.nn.functional.cosine_similarity(r_features.mean(1), h_features.mean(1))

tensor([1.], grad_fn=<SumBackward1>)

In [35]:
roberta.fill_mask('xin <mask> mày một con chó')

[('xin cho mày một con chó', 0.6511508226394653, ' cho'),
 ('xin  mày một con chó', 0.025863606482744217, ' '),
 ('xin tặng mày một con chó', 0.02465791068971157, ' tặng'),
 ('xin chào mày một con chó', 0.016041168943047523, ' chào'),
 ('xin dạy mày một con chó', 0.010530495084822178, ' dạy')]

In [98]:
from transformers import PreTrainedTokenizer, AutoModel
import os.path as osp
import torch
from sentencepiece import SentencePieceProcessor
from fairseq.data.dictionary import Dictionary
from typing import *


class KiBertTokenizer(PreTrainedTokenizer):
    def __init__(self,vocab_file, spm, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert osp.exists(vocab_file), vocab_file
        assert osp.exists(spm), spm
        self.sp = SentencePieceProcessor(spm)
        source_dict = Dictionary()
        self.source_dict = source_dict.load(vocab_file)
        
    @classmethod
    def from_pretrained(cls, path_to_pretrained):
        return cls(osp.join(path_to_pretrained, 'dict.txt'), osp.join(path_to_pretrained, 'bert_spm.model'))
    
    def encode(self, sentence: str, *addl_sentences, no_separator=False, return_tensors='pt') -> torch.LongTensor:
        sp, source_dictionary = self.sp, self.source_dict
        if return_tensors != 'pt':
            raise NotImplemented
        def _encode(x:str):
            return " ".join(
                        sp.Encode(
                            x, out_type=str, enable_sampling=False, alpha=None,
                        )
                    )
        bpe_sentence = "<s> " + _encode(sentence) + " </s>"
        for s in addl_sentences:
            bpe_sentence += " </s>" if not no_separator else ""
            bpe_sentence += " " + _encode(s) + " </s>"
        tokens = source_dictionary.encode_line(
            bpe_sentence, append_eos=False, add_if_not_exist=False
        )
        return tokens.long()
        
    def decode(self, token_ids: torch.LongTensor) -> str:
        token_strs = [self.source_dict[idx] for idx in token_ids]
        return self.sp.DecodePieces(token_strs)

In [99]:
tokenizer = KiBertTokenizer.from_pretrained('../store/vi-roberta-base/')

In [100]:
ids = tokenizer.encode('erfsdfsadf asdf asdf asdf asd')

In [101]:
ht = AutoTokenizer.from_pretrained('roberta-base')

In [102]:
# ht.encode('hello', return_tensors='pt')

Some weights of RobertaModel were not initialized from the model checkpoint at .cache/hf_ckpt_path/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [109]:
# ht.encode('hello')

In [114]:
from transformers import AutoModel
model = AutoModel.from_pretrained('.cache/hf_ckpt_path/')
ids = tokenizer.encode('xin')[None]
features = model(ids).last_hidden_state
print(f'{features.shape=}')

Some weights of RobertaModel were not initialized from the model checkpoint at .cache/hf_ckpt_path/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


features.shape=torch.Size([1, 3, 768])
